Tutorial for image recognition using keras:

https://www.sitepoint.com/keras-face-detection-recognition/

Install necessary libraries:

pip install facenet-pytorch

pip install numpy

pip install pandas

pip install scikit-learn

## First, let's ensure that our image preprocessing was successful, generating

In [ ]:
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

# Initialize MTCNN (for face detection) and InceptionResnetV1 (for facial embeddings)
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Function to generate facial encoding (embedding) for an image
def get_face_encoding(image_path):
    img = Image.open(image_path)
    img_cropped = mtcnn(img)  # Detect faces in the image
    if img_cropped is None:
        return None  # No face detected, return None
    img_embedding = model(img_cropped)  # Extract the facial embedding
    return img_embedding.detach().numpy().flatten()  # Return as 1D array

# Check similarity between two sample images
image_path_1 = "path_to_image_01.jpg"
image_path_2 = "path_to_image_02.jpg"

encoding_1 = get_face_encoding(image_path_1)
encoding_2 = get_face_encoding(image_path_2)

if encoding_1 is not None and encoding_2 is not None:
    similarity = cosine_similarity([encoding_1], [encoding_2])[0][0]
    print(f"Cosine similarity between {image_path_1} and {image_path_2}: {similarity:.4f}")
else:
    print("Face not detected in one or both images.")


No duplicate images found.


### This indicates that no two images in our dataset have the same embedding (facial encoding).

## Let's choose two images that look very similar on which to test this. We will run a cosine similarity test:

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch
import numpy as np

# Initialize MTCNN (for face detection) and InceptionResnetV1 (for facial embeddings)
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Function to generate facial encoding (embedding) for an image
def get_face_encoding(image_path):
    img = Image.open(image_path)
    img_cropped = mtcnn(img)  # Detect faces in the image
    if img_cropped is None:
        return None  # No face detected, return None
    img_embedding = model(img_cropped)  # Extract the facial embedding
    return img_embedding.detach().numpy().flatten()  # Return as 1D array (512-dimensional)

# Example usage
image_path_1 = "path_to_image_01.jpg"
image_path_2 = "path_to_image_02.jpg"

encoding_1 = get_face_encoding(image_path_1)
encoding_2 = get_face_encoding(image_path_2)

# Check if the embeddings have the same dimensionality
print(f"Encoding 1 shape: {encoding_1.shape}")
print(f"Encoding 2 shape: {encoding_2.shape}")

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Example to check similarity between two sample images
encoding_1 = get_face_encoding("/workspace/DS4002Project3/DATA/celebrities_extra/Priyanka-Chopra/017.jpg")
encoding_2 = get_face_encoding("/workspace/DS4002Project3/DATA/celebrities_extra/Priyanka-Chopra/019.jpg")

if encoding_1 is not None and encoding_2 is not None:
    # Ensure that the encodings are the same dimensionality before comparing
    encoding_1 = encoding_1.reshape(1, -1)  # Reshape to 2D if it's 1D
    encoding_2 = encoding_2.reshape(1, -1)  # Reshape to 2D if it's 1D
    similarity = cosine_similarity(encoding_1, encoding_2)[0][0]
    print(f"Cosine similarity between {image_path_1} and {image_path_2}: {similarity:.4f}")
else:
    print("Face not detected in one or both images.")


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 1024 while Y.shape[1] == 512